In [6]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine

In [4]:
connect = psycopg2.connect(
    host = "localhost",
    database = "financial_database",
    user = 'postgres',
    password = 'crack123'
)

In [12]:
cursor = connect.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id varchar primary key,
    customer_id varchar,
    customer_dob date,
    customer_gender char(1),
    customer_location varchar,
    account_balance numeric,
    transaction_date date,
    transaction_time time,
    transaction_amount numeric)
                ''')

connect.commit()
cursor.close()

In [11]:
data = pd.read_csv("/Users/paramjaswal/Desktop/FinancialProject/bank_transactions.csv")
data.head(10)

,transaction_id,customer_id,customer_dob,customer_gender,customer_location,account_balance,transaction_date,transaction_time,transaction_amount
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2002-08-16,143207,25.00
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2002-08-16,141858,27999.00
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2002-08-16,142712,459.00
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2002-08-16,142714,2060.00
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2002-08-16,181156,1762.50
5,T6,C1536588,8/10/72,F,ITANAGAR,53609.20,2002-08-16,173940,676.00
6,T7,C7126560,26/1/92,F,MUMBAI,973.46,2002-08-16,173806,566.00
7,T8,C1220223,27/1/82,M,MUMBAI,95075.54,2002-08-16,170537,148.00
8,T9,C8536061,19/4/88,F,GURGAON,14906.96,2002-08-16,192825,833.00
9,T10,C6638934,22/6/84,M,MUMBAI,4279.22,2002-08-16,192446,289.11


In [16]:
def parse_dates(date_series):
    for fmt in ('%d/%m/%y', '%d/%m/%Y'):
        try:
            return pd.to_datetime(date_series, format=fmt, errors='coerce')
        except (ValueError, TypeError):
            pass
    return pd.to_datetime(date_series, errors='coerce')

data['customer_dob'] = parse_dates(data['customer_dob'])
data['transaction_date'] = parse_dates(data['transaction_date'])

data['transaction_time'] = data['transaction_time'].apply(lambda x: pd.to_datetime(str(int(x)).zfill(6), format='%H%M%S').time() if pd.notnull(x) else None)

In [17]:
engine = create_engine('postgresql+psycopg2://postgres:"YOUR PASSWORD"@localhost/financial_database')


data.to_sql('transactions', engine, if_exists='append', index=False)

567

In [18]:
data.describe()

,customer_dob,account_balance,transaction_date,transaction_amount
count,986874,1.046198e+06,0,1.048567e+06
mean,1990-12-14 20:39:44.751853440,1.154035e+05,NaT,1.574335e+03
min,1969-01-01 00:00:00,0.000000e+00,NaT,0.000000e+00
25%,1983-09-06 00:00:00,4.721760e+03,NaT,1.610000e+02
50%,1988-08-04 00:00:00,1.679218e+04,NaT,4.590300e+02
75%,1992-02-07 00:00:00,5.765736e+04,NaT,1.200000e+03
max,2068-12-31 00:00:00,1.150355e+08,NaT,1.560035e+06
std,NaN,8.464854e+05,NaN,6.574743e+03


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB
